In [3]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_community.tools import ArxivQueryRun, DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama 

In [8]:
def create_research_agent(llm):
    """
    Creates a research agent equipped with Arxiv and DuckDuckGo search tools.

    Args:
        llm: An instance of a LangChain language model.

    Returns:
        An AgentExecutor ready to process research queries.
    """
    # --- 2. Define the Tools ---
    # The agent will have access to two tools:
    # - Arxiv: For searching academic papers specifically on arxiv.org.
    # - DuckDuckGoSearch: For general web searches to find papers on other sites
    #   or to get a broader context.
    arxiv_tool = ArxivQueryRun()
    ddg_search_tool = DuckDuckGoSearchRun()

    tools = [
        Tool(
            name="arxiv_search",
            func=arxiv_tool.run,
            description="Use this tool to search for research papers on arXiv.org. The input should be a search query."
        ),
        Tool(
            name="web_search",
            func=ddg_search_tool.run,
            description="Use this tool for general web searches to find research papers or articles not available on arXiv."
        ),
    ]

    # --- 3. Create the Agent's Prompt ---
    # This prompt acts as the agent's "brain," instructing it on its role,
    # how to use its tools, and how to format its final answer.
    # The ReAct (Reason+Act) framework is used here, which is excellent for
    # tool-using agents.
    prompt_template = """
    You are a diligent and expert research assistant. Your goal is to find relevant research papers on a given topic.
    To answer the user's request, you must follow these steps:
    1.  Start by using the `arxiv_search` tool to see if you can find papers on the primary repository.
    2.  If the arXiv search is not sufficient or you need broader context, use the `web_search` tool.
    3.  Analyze the results from your tools.
    4.  When you have found enough information and are confident in your findings, provide a final answer.
    5.  Your final answer should be a formatted list of the papers you found, including their titles and links.

    You have access to the following tools:
    {tools}

    Use the following format for your reasoning process:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question, which should be a formatted list of papers.

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)

    # --- 4. Create the Agent ---
    # We use the create_react_agent function, which creates an agent that
    # uses the ReAct framework to decide which tool to use based on the
    # user's input and its previous actions.
    agent = create_react_agent(llm, tools, prompt)

    # --- 5. Create the Agent Executor ---
    # The AgentExecutor is the runtime for the agent. It's what actually
    # calls the agent, executes the chosen tools, and passes the results
    # back to the agent for the next reasoning step.
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,  # Set to True to see the agent's thought process
        handle_parsing_errors=True  # Helps with robustness
    )

    return agent_executor

In [9]:
llm = ChatOllama(model="gemma3:12b", temperature=0)

In [10]:
 research_agent = create_research_agent(llm)

In [11]:
topic = "multi-agent systems with orchestrators"

In [12]:
response = research_agent.invoke({"input": topic})



> Entering new AgentExecutor chain...
Okay, let's start by searching arXiv for papers related to "multi-agent systems with orchestrators." This should give me a good starting point.
    
Action: arxiv_search
Action Input: "multi-agent systems orchestrator"Published: 2024-01-12
Title: Intelligent Data-Driven Architectural Features Orchestration for Network Slicing
Authors: Rodrigo Moreira, Flavio de Oliveira Silva, Tereza Cristina Melo de Brito Carvalho, Joberto S. B. Martins
Summary: Network slicing is a crucial enabler and a trend for the Next Generation
Mobile Network (NGMN) and various other new systems like the Internet of
Vehicles (IoV) and Industrial IoT (IIoT). Orchestration and machine learning
are key elements with a crucial role in the network-slicing processes since the
NS process needs to orchestrate resources and functionalities, and machine
learning can potentially optimize the orchestration process. However, existing
network-slicing architectures lack the ability to de

In [13]:
print("\n--- Final Answer ---")
print(response["output"])


--- Final Answer ---
*   **Intelligent Data-Driven Architectural Features Orchestration for Network Slicing**
    *   Authors: Rodrigo Moreira, Flavio de Oliveira Silva, Tereza Cristina Melo de Brito Carvalho, Joberto S. B. Martins
    *   Link: [https://arxiv.org/abs/2401.11774](https://arxiv.org/abs/2401.11774)
*   **Joint$\lambda$: Orchestrating Serverless Workflows on Jointcloud FaaS Systems**
    *   Authors: Jianfei Liu, Rui Li, Zhilin Yang, Peichang Shi, Guodong Yi, Huaimin Wang
    *   Link: [https://arxiv.org/abs/2505.15379](https://arxiv.org/abs/2505.15379)
*   **Live Orchestral Piano, a system for real-time orchestral music generation**
    *   Authors: Léopold Crestel, Philippe Esling
    *   Link: [https://arxiv.org/abs/1705.05779](https://arxiv.org/abs/1705.05779)
